# Bitcoin blocks

This is a Jupyter notebook. We're going to use it to go through some examples of how blocks function in the Bitcoin blockchain.

To run the code in a cell, press shift-enter.

In [ ]:
# First we import everything we need

import helper
from binascii import hexlify, unhexlify

# Cryptographic hashes

Cryptographic hashes have the following properties:

1. it is *infeasible* to generate a message from its hash value (preimage resistance)
2. a small change to a message results in a completely different digest (avalanche effect)
3. it is *infeasible* to find two different messages with the same hash value (collision resistance)

Let's test (1) and (2):

- What's the double sha256 digest of "devplusplus"?
- What's the double sha256 digest of "devplusplut"?
- What's the preimage for the digest "0x707e397fc6c0327b6d9c3a2be68d4fd2456609caa53f8a4fdc7131d3944b516c"?

In [ ]:
def hash(message):
    # hash the block with the nonce
    print("{0:#0{1}x}".format(int.from_bytes(helper.double_sha256((message).encode()), 'big'), 66))

hash(message="devplusplus")

# Proof-of-Work

Proof-of-work determines who gets to build the next block in the Bitcoin blockchain. It relies on the fact that a cryptographic hash function is one-way, and the output is essentially randomly distributed.

We're going to do some manual proof-of-work over a short message.

Find a valid 'block' for the message "devplusplus" with 4 bits of difficulty.

Try running the `validate_block()` function with the same message but different nonces.

In [ ]:
def validate_block(message, nonce, difficulty):
    # hash the block with the nonce
    block_hash = int.from_bytes(helper.double_sha256((message + nonce).encode()), 'big')
    if block_hash < 2 ** (256 - difficulty):
        # winner! Our digest was below the target difficulty
        print("Valid block:    {0:#0{1}x}.\nNonce = {2}".format(block_hash, 66, nonce))
        return True
    else:
        # Sorry, your digest was too large!
        print("Invalid block:  {0:#0{1}x}.\nNonce = {2}".format(block_hash, 66, nonce))
        return False

validate_block(message="devplusplus", nonce="nonce1", difficulty=4)

# Validating work

Proof-of-work is *hard* to do, but *easy* to validate. Once you and your neighbor have a valid block, verify that their block is valid by running `validate_block()` with their nonce.

In [ ]:
validate_block(message="devplusplus", nonce=<put your neighbors nonce here>, difficulty=4)

# Difficulty

Find the target for block 489888 on the Bitcoin main net. Go to [smartbit.com.au](https://www.smartbit.com.au/block/489888) and use the `Bits` field.

Note that the `Bits` field is displayed in Big-endian

In [ ]:
bits = unhexlify('18013ce9')
exponent = bits[0]
coefficient = int.from_bytes(bits[1:], 'big')
target = coefficient*2**(8*(exponent-3))
print("{0:#0{1}x}".format(target, 66))

Find the target difficulty for block 489888 on the Bitcoin mainnet using `difficulty` from [smartbit.com.au](https://www.smartbit.com.au/block/489888).

In [ ]:
base_difficulty = 0x00000000ffff0000000000000000000000000000000000000000000000000000
difficulty = 2
target = int(base_difficulty/difficulty)
print("{0:#0{1}x}".format(target, 66))# Difficulty

# Retarget

The timestamp for block 489888 is 1508040302.
The timestamp for block 491903 is 1509036725.

Calculate what the target should be for block 491904.

In [ ]:
def print_target(target):
    print("{0:#0{1}x}".format(target, 66))

def target_from_bits(bits_in):
    bits = unhexlify(bits_in)
    exponent = bits[0]
    coefficient = int.from_bytes(bits[1:], 'big')
    target = coefficient*2**(8*(exponent-3))
    print_target(target)
    return target

target_489888 = target_from_bits('')
time_489888 =
time_491903 =

new_target = target_489888
new_target *= (time_491903 - time_489888)
new_target /= 14 * 24 * 60 * 60
new_target = int(new_target)

# this is a bit of a hack to get the top bytes from new_target
new_target_top_bytes = new_target.to_bytes((new_target.bit_length() + 7) // 8, 'big')[0:2]
new_target_int = int.from_bytes(new_target_top_bytes, 'big') * 2 ** (new_target.bit_length() - 16)

print_target(new_target_int)
target_from_bits('')

# Total Bitcoin supply

Let's work out how many Bitcoin there will ever be.

We'll work using satoshis in the exercise. There are 100,000,000 satoshi in 1 Bitcoin.

- The block subsidy in the first block was 50 Bitcoin
- The subsidy halves every 210,000 blocks (approx every 4 years)
- The subsidy continues for 32 halvings (approx 128 years)

In [ ]:
INITIAL_SUBSIDY_IN_SATOSHI = # fill in the initial subsidy in Satoshi
BLOCKS_PER_HALVINGS = # fill in the number of blocks in a halving period
NUMBER_OF_HALVINGS = # fill in the number of halvings

total_supply = 0  # There are initially no Bitcoin
block_subsidy = INITIAL_SUBSIDY_IN_SATOSHI
for halving in range(NUMBER_OF_HALVINGS):
    bitcoin_in_halving = block_subsidy * BLOCKS_PER_HALVINGS  # number of new Bitcoin mined in this halving period
    total_supply += bitcoin_in_halving  # total supply after this halving period
    print("After halving {}, there are {} Bitcoin".format(halving + 1, total_supply / 10**8))
    block_subsidy >>= 1  # halve the reward!


# Blocks

We can work with raw blocks using the Block class.

In [ ]:
from block import Block
from io import BytesIO

raw_block = BytesIO(unhexlify('010000006fe28c0ab6f1b372c1a6a246ae63f74f931e8365e15a089c68d6190000000000982051fd1e4ba744bbbe680e1fee14677ba1a3c3540bf7b1cdb606e857233e0e61bc6649ffff001d01e362990101000000010000000000000000000000000000000000000000000000000000000000000000ffffffff0704ffff001d0104ffffffff0100f2052a0100000043410496b538e853519c726a2c91e61ec11600ae1390813a627c66fb8be7947be63c52da7589379515d4e0a604f8141781e62294721166bf621e73a82cbf2342c858eeac00000000'))
block_obj = Block.parse(raw_block)
print("Block hash is {}".format(hexlify(block_obj.hash())))
print("Difficulty is {}".format(block_obj.difficulty()))
print("Timestamp is {}".format(block_obj.timestamp))
